In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
###############################################################################
### Inverse Beta Decay (IBD) kinematics and cross-section calculator ##########
### Based on Phys.Rev.D60 (1999) 053003 (arxiv:hep-ph/9903554) ################
###############################################################################

# --- Generates the 'junoenuspec.npy' and 'junosigspec.npy' files --- #

# Cosθ as a function of Enu and Epos and its derivative with respect to Epos 
# Computes cos(theta) for a given antineutrino energy Enu and positron energy Epos, along with the derivative dcosθ/dEpos.
def cost_(Enu,Epos):
    Mn = 939.56542052 # Neutron mass [MeV]
    Mp = 938.27208816 # Proton mass [MeV]
    Me = 0.51099895000 # Electron mass [MeV]
    M = (Mn+Mp)/2
    Delta = Mn - Mp
    y = (Delta**2 - Me**2)/2
    Ee0 = Enu - Delta
    ve0 = np.sqrt(1 - Me**2/Ee0**2)
    
    cost = 1/ve0*(1 - M/Enu*(1-(Epos + y/M)/Ee0))
    dcostdepos = M/(Ee0*Enu*ve0)
    
    return cost, dcostdepos

In [ ]:
# Epos as a function of Enu and cosθ and its derivative with respect to cosθ
# Computes the positron energy Epos and the derivative dEpos/dcosθ for given a antineutrino energy Enu and cosθ.
def Ee(Enu, cost):
    Mn = 939.56542052
    Mp = 938.27208816
    Me = 0.51099895000
    M = (Mn+Mp)/2
    Delta = Mn - Mp
    y = (Delta**2 - Me**2)/2
    Ee0 = Enu - Delta
    ve0 = np.sqrt(1 - Me**2/Ee0**2)
    
    Epos = Ee0*(1 - Enu/M*(1 - ve0*cost)) - y/M
    deposdcost = Ee0*Enu*ve0/M
    
    return Epos, deposdcost

In [ ]:
# Differential cross-section dσ/dEpos
# Computes the differential IBD cross-section dσ/dEpos [cm^2] for a given antineutrino energy Enu and positron energy Epos.
def cross_IBD(Enu,Epos):
    
    # Physical constants
    Mn = 939.56542052
    Mp = 938.27208816
    Me = 0.51099895000
    M = (Mn+Mp)/2
    Delta = Mn - Mp
    
    # Kinematics
    Ee0 = Enu - Delta
    pe0 = np.sqrt(Ee0**2 - Me**2)
    ve0 = np.sqrt(1 - Me**2/Ee0**2)

    # Couplings and corrections
    f = 1
    f2 = 3.706
    g = 1.26
    G = 1.1663787  # MeV^-2 (factor 1e-11 suppressed)
    Deltainn = 0.024
    ctc = 0.974
    sigma0 = (G**2*ctc**2)/np.pi*(1 + Deltainn) # MeV^-4
    
    # Compute cosθ and its derivative
    cost, dcostdepos = cost_(Enu, Epos)
    
    # Gamma term from paper
    gamma = 2*(f+f2)*g*((2*Ee0 + Delta)*(1 - ve0*cost) - Me**2/Ee0) + \
    (f**2+g**2)*(Delta*(1 + ve0*cost) + Me**2/Ee0) + \
    (f**2+3*g**2)*((Ee0+Delta)*(1-cost/ve0)-Delta) + \
    (f**2-g**2)*((Ee0+Delta)*(1-cost/ve0)-Delta)*ve0*cost

    # If below threshold, σ = 0
    if Epos < Me:
        sigepos=0
    else:
        ve1 = np.sqrt(1 - Me**2/Epos**2)
        pe1 = np.sqrt(Epos**2 - Me**2)
        
        # dσ/dcosθ
        dsigdcost = sigma0/2*((f**2+3*g**2)+(f**2-g**2)*ve1*cost)*Epos*pe1 - \
        sigma0/2*gamma/M*Ee0*pe0
        
        # dσ/dEpos
        sigepos = dsigdcost*dcostdepos

    # Unit conversion MeV^-2 -> cm^2 (plus factor 1e-22 suppressed in paper)
    mevtocm=(1e6*5.068e4)**2.
    sigepos=sigepos/mevtocm*1e-22

    return sigepos

In [ ]:
# Effective cross-section per bin of reconstructed energy
# Computes an effective cross-section in each bin of reconstructed visible energy.
def CS_bin():

    # Physical constants
    Mn = 939.56542052
    Mp = 938.27208816
    Me = 0.51099895000
    Delta = Mn - Mp

    # Bins in reconstructed energy
    Ebins = np.linspace(0.88,10,229)

    enubins=[]
    sigbins=[]
    
    # Loop over reconstructed energy bins
    for i in range(0,len(Ebins)-1):
        ebinmin = Ebins[i]
        ebinmax = Ebins[i+1]
        ebinlist = np.linspace(ebinmin,ebinmax,51)
        
        # Energy resolution
        res=(0.5*(ebinmin+ebinmax))*np.sqrt(0.0261**2/(0.5*(ebinmin+ebinmax)) + 0.0064**2 + 0.012**2/(0.5*(ebinmin+ebinmax))**2)
        
        # Kinematic limits
        evmin = ebinmin - 5*res
        if evmin < 1.0218492715644438:
            evmin = 1.0218492715644438
        evmax = ebinmax + 5*res
        epmin = evmin - Me
        if epmin < 0.510850321564444:
            epmin = 0.510850321564444
        epmax = evmax - Me
        enumin = Delta + epmin
        if enumin < 1.806:
            enumin = 1.806
        enumax = Delta + epmax
        enulist = np.linspace(enumin,enumax,26)

        # Integrating over Enu and Epos
        sigenutot = []
        sig4 = []
        for Enu in enulist:
            Eposmin,deposdcost=Ee(Enu,-1)
            Eposmax,deposdcost=Ee(Enu, 1)
            Eposlist = np.linspace(Eposmin,Eposmax,51)
            sigenu=[]
            for Epos in Eposlist:
                sigenu.append(cross_IBD(Enu,Epos))
                
            Evmin = Eposmin + Me
            Evmax = Eposmax + Me
            Evlist = np.linspace(Evmin, Evmax, 51)
            sig3 = []
            for erec in ebinlist:
                sigma = Evlist*np.sqrt(0.0261**2/Evlist + 0.0064**2 + 0.012**2/Evlist**2)
                resol = np.exp(-0.5 * ((erec - Evlist) / sigma) ** 2.)/np.sqrt(2.*np.pi)/sigma
                sig2=resol*sigenu
                sig3.append(np.trapz(sig2, Eposlist))
            sig4.append(np.trapz(sig3,ebinlist))
        enubins.append(enulist)
        sigbins.append(sig4)

    # Saving
    np.save('junoenuspec.npy',enubins)
    np.save('junosigspec.npy',sigbins)
    
    return 0

CS_bin()